In [49]:
import cython
import numpy as np
import TidalPy
print(TidalPy.__version__)

0.5.0a5.dev10


In [50]:
import matplotlib.pyplot as plt

def plot_ys(radius_array, *ys):
    
    num_ys = len(ys)
    
    if num_ys == 2:
        ncols = 2
        nrows = 1        
    elif num_ys == 4:
        ncols = 2
        nrows = 2
    else:
        ncols = 3
        nrows = 2
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols)
    
    for i in range(nrows):
        for j in range(ncols):
            k = (i * ncols) + j
            axes[i, j].plot(radius/1000, ys[k])
    plt.show()

In [51]:
import numpy as np
from scipy.constants import G

from TidalPy.rheology.complex_compliance.compliance_models import newton, elastic, maxwell
from TidalPy.radial_solver.numerical.initial import find_initial_guess
from TidalPy.utilities.spherical_helper import calculate_mass_gravity_arrays

radius = np.linspace(1., 6.371e6, 50)
density = np.ones_like(radius) * 5515.0
shear_modulus = np.zeros_like(radius)
viscosity = 1000. * np.ones_like(radius)
bulk_modulus = np.ones_like(radius) * 100.0e9
G_to_use = G
order_l = 2
frequency = 2 * np.pi / (24 * 60 * 60)
complex_shear = newton(frequency, 0., viscosity)**(-1)

volume_array, mass_array, gravity = calculate_mass_gravity_arrays(radius, density, G_to_use)

init_guess = find_initial_guess(
        is_solid=False, is_static=True, is_incompressible=False, is_kamata=True,
        radius=radius[0], shear_modulus=complex_shear[0], bulk_modulus=bulk_modulus[0],
        density=density[0], frequency=frequency, order_l=order_l, G_to_use=G_to_use
    )

solutions = list()
for sol in init_guess:
    num_y = sol.size
    y0 = np.zeros(num_y*2, dtype=np.float64, order='C')
    for i in range(num_y):
        y0[2 * i] = np.real(sol[i])
        y0[2 * i + 1] = np.imag(sol[i])
    solutions.append(y0)

In [52]:
solutions

[array([1.000e+00, 0.000e+00, 2.000e+00, 0.000e+00])]

In [53]:
rtols = np.ones(solutions[0].size, dtype=np.float64, order='C') * 1.0e-6
atols = np.ones(solutions[0].size, dtype=np.float64, order='C') * 1.0e-6
from TidalPy.radial_solver.numerical.derivatives.odes_x import LiquidStaticCompressible, LiquidDynamicCompressible

complex_shear_zeros = np.zeros_like(complex_shear)
SolverInst = LiquidDynamicCompressible(
    radius, density, gravity, complex_shear_zeros, bulk_modulus, frequency, order_l, G_to_use,
    t_span=(radius[0], radius[-1]), y0=solutions[0], rtol=1.0e-7, atol=1.0e-8, rtols=None, atols=None,
    rk_method=1, auto_solve=False
)

In [ ]:
SolverInst.solve()
print(SolverInst.success)
print(SolverInst.message)

In [ ]:
SolverInst.message

In [32]:
solutions[0]

array([1.000e-02, 0.000e+00, 2.000e-01, 0.000e+00])

In [2]:
%load_ext cython

In [ ]:
%%cython --annotate --force
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

from 



In [9]:
%%cython --annotate --force
# distutils: language = c++

from libcpp cimport bool as cpp_bool_t
import numpy as np
cimport numpy as np

from scipy.constants import G as G_
from TidalPy.radial_solver.nondimensional import non_dimensionalize_physicals, re_dimensionalize_radial_func
from TidalPy.radial_solver.numerical.collapse import collapse_solutions
from TidalPy.radial_solver.numerical.initial import find_initial_guess
from TidalPy.radial_solver.numerical.interfaces import find_interface_func

# Import cythonized functions
from TidalPy.radial_solver.numerical.interfaces.interfaces_x cimport find_solution_num


# Setup globals
cdef double G
G = G_


def radial_solver_x(double[:] radius_array,
                    double complex[:] complex_shear_array,
                    double[:] bulk_array,
                    double[:] density_array,
                    double[:] gravity_array,
                    double frequency,
                    double planet_bulk_density,
                    tuple is_solid_by_layer,
                    tuple is_static_by_layer,
                    tuple is_incompressible_by_layer,
                    tuple indices_by_layer,
                    unsigned int degree_l = 2, 
                    double complex[:] surface_boundary_conditions = None,
                    cpp_bool_t solve_load_numbers = False,
                    cpp_bool_t use_kamata = False,
                    int integration_method = 1,
                    double integration_rtol = 1.0e-8,
                    double integration_atol = 1.0e-9, 
                    cpp_bool_t verbose = False,
                    cpp_bool_t nondimensionalize = True,
                    ):
    # General indexing
    cdef Py_ssize_t i, j, filled_j
    
    # Pull out key information
    cdef double radius_planet
    cdef Py_ssize_t num_layers, num_interfaces, num_radius
    radius_planet  = radius_array[-1]
    # TODO: Exception to check if there is at least one layer?
    num_layers     = len(is_solid_by_layer)
    num_interfaces = num_layers - 1
    num_radius     = len(radius_array)
    
    # Non-dimensionalize inputs
    cdef double G_to_use
    
    if nondimensionalize:
        radius_array, gravity_array, density_array, complex_shear_array, bulk_array, frequency, G_to_use = \
            non_dimensionalize_physicals(
                    radius_array, gravity_array, density_array, complex_shear_array, bulk_array, frequency,
                    mean_radius=radius_planet, bulk_density=planet_bulk_density
                    )
    else:
        G_to_use = G
    
    # Find boundary condition at the top of the planet -- this is dependent on the forcing type.
    #     Tides (default here) follow the (y2, y4, y6) = (0, 0, (2l+1)/R) rule
    cdef np.ndarray[np.complex128_t, ndim=1] bc_array
    cdef double complex[:] bc_array_view
    bc_array = np.empty(3, dtype=np.complex128)
    bc_array_view = bc_array
    
    if surface_boundary_conditions is None:
        # Assume tides
        for i in range(3):
            if i == 2:
                if nondimensionalize:
                    bc_array_view[i] == (2. * degree_l + 1.) / 1.
                else:
                    bc_array_view[i] == (2. * degree_l + 1.) / radius_planet
    else:
        # Use user input
        # TODO: Exception if user input is wrong size.
        for i in range(3):
            bc_array_view[i] = surface_boundary_conditions[i]
    
    # Find number of solutions per layer
    cdef np.ndarray[np.uint16_t, ndim=1] num_solutions_by_layer_array
    cdef unsigned short num_solutions_by_layer_view
    num_solutions_by_layer_array = np.zeros(num_layers, dtype=np.ushort)
    num_solutions_by_layer_view = num_solutions_by_layer_array
    
    cdef np.ndarray[np.uint8_t, ndim=2] indices_by_layer_array
    cdef unsigned char[:, :] indices_by_layer_view
    indices_by_layer_array = np.zeros((num_layers, num_radius), dtype=np.uint8)
    indices_by_layer_view = indices_by_layer_array
    
    cdef Py_ssize_t num_radius_in_layer
    cdef np.ndarray[np.uint32_t, ndim=1] layer_length_array
    cdef unsigned int[:] layer_length_view
    layer_length_array = np.zeros(num_layers, dtype=np.uint32)
    layer_length_view = layer_length_array
    
    cdef cpp_bool_t layer_is_solid, layer_is_static, layer_is_incomp
    
    for i in range(num_layers):
        
        layer_is_solid = is_solid_by_layer[i]
        layer_is_static = is_static_by_layer[i]
        layer_is_incomp = is_incompressible_by_layer[i]
        
        num_solutions_by_layer_array[i] = find_solution_num(layer_is_solid, layer_is_static, layer_is_incomp)
        
        # Determine which radii fall within this layer
        num_radius_in_layer = 0
        for j in range(num_radius):
            if indices_by_layer[i][j] == 1:
                indices_by_layer_view[i, j] = 1
                num_radius_in_layer += 1
        layer_length_view[i] = <unsigned int>num_radius_in_layer
                
        
    
    # Store solutions per layer.
    # TODO: Make this a ndarray?
    cdef list solutions_by_layer_list
    solutions_by_layer_list = list()
    
    # Other variables
    cdef np.ndarray[np.float64_t, ndim=1] layer_radii_array, layer_bulk_array, layer_density_array, layer_gravity_array
    cdef np.ndarray[np.complex128_t, ndim=1] layer_shear_array
    cdef double[:] layer_radii_view, layer_bulk_view, layer_density_view, layer_gravity_view
    cdef double complex[:] layer_shear_view
    
    cdef np.ndarray[np.complex128_t, ndim=2] solutions_below_array
    cdef double complex[:, :] solutions_below_view
    
    cdef double layer_radius_lower, layer_radius_upper
    cdef double layer_gravity_lower, layer_gravity_upper
    cdef double layer_density_lower, layer_bulk_lower
    cdef double complex layer_shear_lower
    
    cdef Py_ssize_t n_layer_radius
    cdef unsigned short layer_num_solutions
    
    # # Main Solution Loop
    # Ready to solve the viscoelastic-gravitational problem for each layer, obtaining multiple solutions per layer
    #  which will later be collapsed via a linear combination (subjected to boundary conditions) into a single
    #  solution for the entire planet.
    
    # Loop through layers from bottom to top and solve the differential equations
    for i in range(num_layers):
        
        # Pull out layer information
        num_radius_in_layer = <Py_ssize_t>layer_length_view[i]
        # Build arrays filled with this layer's properties
        layer_radii_array   = np.empty(num_radius_in_layer, dtype=np.float64)
        layer_bulk_array    = np.empty(num_radius_in_layer, dtype=np.float64)
        layer_density_array = np.empty(num_radius_in_layer, dtype=np.float64)
        layer_gravity_array = np.empty(num_radius_in_layer, dtype=np.float64)
        layer_shear_array   = np.empty(num_radius_in_layer, dtype=np.complex128)
        layer_radii_view    = layer_radii_array
        layer_bulk_view     = layer_bulk_array
        layer_density_view  = layer_density_array
        layer_gravity_view  = layer_gravity_array
        layer_shear_view    = layer_shear_array
        filled_j = 0
        for j in range(num_radius):
            if indices_by_layer_view[i, j] == 1:
                layer_radii_view[filled_j]   = radius_array[j]
                layer_bulk_view[filled_j]    = bulk_array[j]
                layer_density_view[filled_j] = density_array[j]
                layer_gravity_view[filled_j] = gravity_array[j]
                layer_shear_view[filled_j]   = complex_shear_array[j]
                filled_j += 1
        
        # Get properties at the base of the layer
        layer_shear_lower   = layer_shear_view[0]
        layer_bulk_lower    = layer_bulk_view[0]
        layer_density_lower = layer_density_view[0]
        layer_gravity_lower = layer_gravity_view[0]
        layer_radius_lower  = layer_radii_view[0]
        
        # Get properties at the top of the layer
        layer_radius_upper  = layer_radii_view[num_radius_in_layer - 1]
        layer_gravity_upper = layer_gravity_view[num_radius_in_layer - 1]
        
        # Get physical state of layer
        layer_is_solid      = is_solid_by_layer[i]
        layer_is_static     = is_static_by_layer[i]
        layer_is_incomp     = is_incompressible_by_layer[i]
        
        # Get integration information for layer
        layer_num_solutions = num_solutions_by_layer_array[i]
        
        # Find initial conditions at bottom of layer
        if i == 0:
            # Use initial condition function
            find_initial_guess(
                        layer_is_solid, layer_is_static, layer_is_incomp, use_kamata,
                        layer_radius_lower, layer_shear_lower, layer_bulk_lower, layer_density_lower, frequency,
                        order_l=degree_l, G_to_use=G_to_use
                        )
        else:
            # Use results at the top of the layer below and a interface function
            pass
            
            
            
    
            
            
        
    


Error compiling Cython file:
------------------------------------------------------------
...
        # Find initial conditions at bottom of layer
        if i == 0:
            # Use initial condition function
            find_initial_guess(
                        layer_is_solid, layer_is_static, layer_is_incomp, use_kamata,
                        layer_radius_lower, shear_modulus[0], bulk_modulus[0], density[0], frequency,
                                            ^
------------------------------------------------------------

C:\Users\joepr\.ipython\cython\_cython_magic_87277e562a4163a37047c1ee421551f5937e9b7b.pyx:202:44: undeclared name not builtin: shear_modulus

Error compiling Cython file:
------------------------------------------------------------
...
        # Find initial conditions at bottom of layer
        if i == 0:
            # Use initial condition function
            find_initial_guess(
                        layer_is_solid, layer_is_static, layer_is_incomp, u

In [23]:
x = np.linspace(0., 3., 4)
y = x <= 2.
print(x, y)
f(y)

[0.000e+00 1.000e+00 2.000e+00 3.000e+00] [ True  True  True False]
Yes!
Yes!
Yes!
No :(
